# case #1 : real, pair-wise different eigenvalues of diagonalizable $A$

- $\sigma(A) = [\lambda_1, \ldots, \lambda_n]  \in \mathbb{R}^n$
- $\lambda_1 > \lambda_2 > \ldots > \lambda_n \in \mathbb{R}$
- assume two "views" onto the latent system via $A, \tilde{A}$, $Q, \tilde{Q}$ etc.
- without loss of generality , assume $\tilde{A} = \Lambda = \mbox{diag}(\lambda_1, \ldots, \lambda_n) \in \mathbb{R}^{n \times n}$

What can we learn from the latent state matrices $A, \tilde{A}$, and $Q, \tilde{Q}$ alone, i.e. not looking at $C, \tilde{C}$?

- what we have: $A, \Lambda$ are similar ($A=M \Lambda M^{-1}$), $Q$ and $\tilde{Q}$ are linked via the same $M$ (through $Q = M \tilde{Q} M^\top$)

Results: 

- $A=M \Lambda M^{-1} = WS\Lambda (WS) ^{-1}$ allows identifying $M=WS$ only up to a matrix $S \in \mathbb{R}^{n \times{} n}$
- **restrictions** on $S$ are key now. Seemingly, $S = \mbox{diag}(s_1, \ldots, s_n)$ is diagonal - **not entirely sure yet why**
- not knowing the per-latent scales $s_i \in \mathbb{R}$ is even worse than only not knowing the correct sign   
- latent covariances $Q, \tilde{Q}$ resolve the scales $|s_i|$ 
- $Q, \tilde{Q}$  also resolve the signs $\mbox{sign}(s_i)$ assuming they are not diagonal  

The code belows serves to play out those conditions numerically 

In [ ]:
import numpy as np
from scipy import stats as stats

p, n = 5, 4

M = np.random.normal(size=(n,n)) # change of basis (actually only has to be invertible)

# system matrices in first (=standard) coordinate system
D = np.sort(np.random.normal(size=n))        # draw real, pairwise different eigenvalues 
L = np.diag(D)
Q = stats.wishart.rvs(df=n, scale=np.eye(n)) 
C = np.random.normal(size=(p,n))

# system matrices in second coordinate system
Ar = M.dot(L).dot(np.linalg.inv(M))
Qr = M.dot(Q).dot(M.T)
Cr = C.dot(np.linalg.inv(M))

# recovering original matrices by explicit rotations using established conditions
Lh, Wh = np.linalg.eig(Ar)
idx = np.argsort(Lh)  
Lh, Wh = Lh[idx], Wh[:, idx]

Winv = np.linalg.inv(Wh)
SQS = Winv.dot(Qr).dot(Winv.T)
s = np.sqrt( np.diag(SQS) / np.diag(Q) ) 

signs = (s*Q*s.reshape(-1,1)) / SQS # all pairwise signs, simple reconstruction approach

s *= signs[0,:] # lazy, just align all signs according to first variable

print('compare each entry of real nxn base-change matrix M with reconstruction W*S via M ./ (W*S) : ')
print(M / Wh.dot(np.diag(s)))

# case #2 : complex, pair-wise different eigenvalues of (almost-)general $A$ 
("almost-general" : $A$ diagonalizable over $\mathbb{C}$, which apparently is all matrices except for some set with zero measure)

- $\sigma(A) = [\lambda_1, \ldots, \lambda_n] \in \mathbb{C}^n$
- $ |\lambda_1 | = | \lambda_2 | > | \lambda_4 | = | \lambda_3 | > \ldots > | \lambda_{n_c-1} | = |\lambda_{n_c} | \in \mathbb{R}$  for $n_c \leq n$ many clomplex EVs
- $ |\lambda_{n_c+1} | > | \lambda_{n_c + 2} | > \ldots >  |\lambda_{n} | \in \mathbb{R}$ purely real eigenvalues if $n_c < n$
- again assume two "views" onto the latent system via $A, \tilde{A}$, $Q, \tilde{Q}$ etc.
- without loss of generality , assume $\tilde{A} = \Lambda = \mbox{diag}(\lambda_1, \ldots, \lambda_n) \in \mathbb{C}^{n \times n}$
- working with complex matrices ($\Lambda$ in general, and depending on $M$ also $A$, $Q$)

What can we learn from the latent state matrices $A, \tilde{A}$, and $Q, \tilde{Q}$ alone, i.e. not looking at $C, \tilde{C}$?

- what we have: $A, \Lambda$ are similar ($A=M \Lambda M^{-1}$), $Q$ and $\tilde{Q}$ are linked via the same $M$ (through $Q = M \tilde{Q} M^\top$)

Results: 
- numerically, seems to work just as case with purely real eigenvalues
- oddly, seems to be sensitive to ordering of eigenvalues also within the complex-conjugate EV pairs - **why**?
Corresponding eigenspaces are 2D, no? That is, ordering (and actual direction) of eigenvectors is not meaningful ...

The code belows serves to play out those conditions numerically 

In [ ]:
import numpy as np
from scipy import stats as stats

p, n = 5, 20
nr = 10 # number of real eigenvalues

# system matrices in first (=standard) coordinate system
eig_m_r, eig_M_r, eig_m_c, eig_M_c = -0.9, 0.99, 0.9, 0.99
nc, nc_u = n - nr, (n - nr)//2
assert nc_u * 2 == nc 
ev_r = np.linspace(eig_m_r, eig_M_r, nr)
ev_c = np.exp(2 * 1j * np.pi * np.random.vonmises(mu=0, kappa=1000, size=nc_u))
ev_c = np.linspace(eig_m_c, eig_M_c, (n - nr)//2) * ev_c
M, D = np.zeros((n,n), dtype=complex), np.zeros(n, dtype=complex) # if we want real A, we need to construct M along D
D[:nr], M[:,:nr] = ev_r, np.random.normal(size=(n,nr))        # (normalized) random base change for real eivenvalues
M[:,:nr] /= np.sqrt((M[:,:nr]**2).sum(axis=0)).reshape(1,nr)
ev_c_r, ev_c_c = np.real(ev_c), np.abs(np.imag(ev_c))
V = np.random.normal(size=(n,n))                              # base change for complex EVs has to be adjusted for real A
for i in range(nc_u):
    Vi = V[:,i*2:(i+1)*2] / np.sqrt( np.sum(V[:,i*2:(i+1)*2]**2) )
    M[:,nr+2*i+1], M[:,nr+2*i] = Vi[:,0]+1j*Vi[:,1], Vi[:,0]-1j*Vi[:,1] 
    D[nr+2*i+1], D[nr+2*i] = ev_c_r[i]+1j*ev_c_c[i], ev_c_r[i]-1j*ev_c_c[i]

Q = np.atleast_2d(stats.wishart(5*n, np.eye(n)).rvs()/n)
C = np.random.normal(size=(p,n))

# system matrices in second coordinate system
Ar = M.dot(np.diag(D)).dot(np.linalg.inv(M))
assert np.allclose(Ar, np.real(Ar))
Ar = np.real(Ar)
Qr = M.dot(Q).dot(M.T)
Cr = C.dot(np.linalg.inv(M))

# recovering original matrices by explicit rotations using established conditions
Lh, Wh = np.linalg.eig(Ar)
idx_r = np.where(np.abs(np.imag(Lh)) < 1e-10)[0]
idx_r = idx_r[np.argsort(np.real(Lh[idx_r]))]
idx_c = np.where(np.abs(np.imag(Lh)) >= 1e-10)[0]
idx_c = idx_c[np.argsort(np.abs(Lh[idx_c]))]
idx_c = np.hstack([idx_c[2*i:2*(i+1)][np.argsort(np.imag(Lh[idx_c[2*i:2*(i+1)]]))] for i in range(len(idx_c)//2)] )
idx = np.hstack((idx_r, idx_c))
Lh, Wh = Lh[idx], Wh[:, idx]

Winv = np.linalg.inv(Wh)
SQS = Winv.dot(Qr).dot(Winv.T)
s = np.sqrt( np.diag(SQS) / np.diag(Q) ) 

signs = (s*Q*s.reshape(-1,1)) / SQS # all pairwise signs, simple reconstruction approach

s *= signs[0,:] # lazy, just align all signs according to first variable

print('compare each entry of real nxn base-change matrix M with reconstruction W*S via M ./ (W*S) : ')
assert np.allclose(M / Wh.dot(np.diag(s)), np.real(M / Wh.dot(np.diag(s))))
print(np.real(M / Wh.dot(np.diag(s))))